<a href="https://colab.research.google.com/github/visha1Sagar/NUST_Bank_Chatbot/blob/main/QA_Extraction_Initial_finetuning_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-genai

In [ ]:
from google.colab import userdata
Gemini_Api_Key = userdata.get('GEMINI_API_KEY')

## Getting structured output from Gemini

In [ ]:
from google import genai

prompt = """
Generate Valid Questions and Answer pairs in JSON format from this : {sheet_text}.

Use this JSON schema:

Question_Answer_Pair = {{'question': str, 'answer': str}}
Return: list[Question_Answer_Pair]
"""

client = genai.Client(api_key=Gemini_Api_Key)

def get_qa_pairs(sheet_text):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt.format(sheet_text=sheet_text)
    )

    return response.text

## Reading Data

In [ ]:

import pandas as pd
data = pd.ExcelFile("/content/NUST Bank-Product-Knowledge.xlsx")


In [ ]:
all_qa_pairs = []

In [ ]:
import json

for sheet_name in data.sheet_names:

    # Use the parse method to load the sheet into a DataFrame
    sheet = data.parse(sheet_name)
    sheet_text = sheet.to_string()  # Convert the sheet data to string
    qa_pairs_str = get_qa_pairs(sheet_text)
    try:
        qa_pairs = json.loads(qa_pairs_str.replace("```", "").replace("json", ""))
        all_qa_pairs.extend(qa_pairs)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for sheet {sheet_name}: {e}")
        print(f"Problematic response: {qa_pairs_str}")



In [ ]:
len(all_qa_pairs)

414

In [ ]:
all_qa_pairs

## Saving to the JSON

In [ ]:
import json

with open('qa_pairs.json', 'w') as f:
    json.dump(all_qa_pairs, f, indent=4)
